# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [1]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [0]:
# --- Select Tensorflow 2.0 (only in Google Colab)
%tensorflow_version 2.x

# Environment

Use the following lines to import any needed libraries:

In [0]:
import os, numpy as np
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers

# Data

As in the tutorial, data for this assignment will be simulated via a Python generator:

In [0]:
def Generator(batch_size=128):
    """
    Method to define a Python generator for training data
    
    """
    # --- Define lambda function for random values [-0.5, +0.5]
    lo = -0.5
    hi = +0.5
    rand = lambda shape : np.random.rand(*shape) * (hi - lo) + lo
    
    # --- Define lambda function for linear transform
    m = np.array([0.2, 0.8, -0.3, 0.4, -0.5]).reshape(5, 1)
    b = 0.7
    f = lambda x : np.matmul(x, m) + b + rand((x.shape[0], 1))
    
    while True:
        
        x = np.random.rand(batch_size, 5)
        y = f(x)
        
        yield x, y

**Imporant**: do *not* change the code for the above Generator to ensure that your results are consistent.

# Training

In this assignment we will train a multiple (multivariable) linear regression model, extending the single variable linear regression model in the tutorial The provided input to the model will comprise of a five-element vector, and the output will be a single-element numeric prediction.  

### Define the model

In [6]:
# --- Define model
x = Input(shape=(5,))
op = layers.Dense(1)
y = op(x)
model = Model(inputs=x, outputs=y)

print(x.shape)
print(y.shape)


(None, 5)
(None, 1)


### Compile the model

In [0]:
# --- Define loss and optimizer
loss = losses.MeanSquaredError()
optimizer = optimizers.Adam(learning_rate=1e-3)

# --- Compile model
model.compile(optimizer=optimizer, loss=loss)

### Train the model

In [8]:
model.fit_generator(
    generator = Generator(),
    steps_per_epoch= 500, 
    epochs=10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
500/500 [==============================] - 1s 2ms/step - loss: 0.2965
Epoch 2/10
500/500 [==============================] - 1s 2ms/step - loss: 0.1610
Epoch 3/10
500/500 [==============================] - 1s 2ms/step - loss: 0.1051
Epoch 4/10
500/500 [==============================] - 1s 2ms/step - loss: 0.0886
Epoch 5/10
500/500 [==============================] - 1s 2ms/step - loss: 0.0845
Epoch 6/10
500/500 [==============================] - 1s 2ms/step - loss: 0.0832
Epoch 7/10
500/500 [==============================] - 1s 2ms/step - loss: 0.0835
Epoch 8/10
500/500 [==============================] - 1s 2ms/step - loss: 0.0829
Epoch 9/10
500/500 [==============================] - 1s 2ms/step - loss: 0.0838
Epoch 10/10
500/500 [==============================] - 1s 2ms/step - loss: 0.0836


# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on a batch (or several batches) of data. 

In [0]:
def TestData(batch_size=128):
    """
    Method to define a Python generator for training data
    
    """
    # --- Define lambda function for random values [-0.5, +0.5]
    lo = -0.5
    hi = +0.5
    rand = lambda shape : np.random.rand(*shape) * (hi - lo) + lo
    
    # --- Define lambda function for linear transform
    m = np.array([0.2, 0.8, -0.3, 0.4, -0.5]).reshape(5, 1)
    b = 0.7
    f = lambda x : np.matmul(x, m) + b + rand((x.shape[0], 1)) 

    x = np.random.rand(batch_size, 5)
    y = f(x)
        
    return x, y

In [51]:
#model.layers[1].get_weights()
batch_size = 5000
a,b = TestData(batch_size)
b_p = model.predict(a)
loss = sum((b_p - b)*(b_p-b))/batch_size
#print(loss)

[0.08177169]
